# Multiple Object Tracking

Multi-Object Tracking (MOT) is a core visual ability that humans poses to perform kinetic tasks and coordinate other tasks. The AI community has recognized the importance of MOT via a series of [competitions](https://motchallenge.net). 

 The ability to reason even in the absence of perception input task was highlighted in Lecture 1 using a document camera and a canopy type of occlusion where an object moves below it. In this assignment, the object class is `ball` and the ability to reason over time will be demonstrated using [Kalman Filters](https://en.wikipedia.org/wiki/Kalman_filter). There will be two cases of occlusion: occlusion by a different object and occlusion by the same object (typical case of the later is on tracking people in crowds). 

## Task 1: Understand the problem and setup environment (20 points)

The problem is best described using this explanatory video below of the raw source files of this assignment:

1. [Single object tracking](https://github.com/sseshadr/auvsi-cv-all/blob/master/objectTracking/examples/ball.mp4)
2. [Multi-object tracking](https://github.com/sseshadr/auvsi-cv-all/blob/master/objectTracking/examples/multiObject.avi)

```{eval-rst}
.. youtube:: 0jAC9sMQQuM
```

The associated to the video github is [here](https://github.com/sseshadr/auvsi-cv-all). 






# Write up


Object Detection is a computer vision task where segments of images or videos are taken and analyzed for the presence and precise location of certain instances of objects. On the other hand, object tracking is the abiity to predict the path and trajectory of an object in a video. In multi-object tracking, multiple objects will have to be tracked concurrently. 
The ability to track objects over time will rely on a set of measurements of the position of objects, however, when these measurements are sparsely missing, we need a way to reason about the position of the object over time. This can be done using Kalman filters, which is an algorithm that can be used to give estimates for unknown variabes given a set of recorded measurements. Kalman filters are recursive, and rely on keeping a state of the object which includes variables such as position, velocity, and acceleration. This state is adjusted over time as new measurements become available, but when new measurments are not readily available, Kalman filters can make predictions using this stored state. 

To demonstrate the ability for Kalman Filters to reason over time, they will be used to predict the trajectories of sports balls. In the first part of the assignment, we will be implementing an object detector using the OpenCV library and YOLOv3 model. Individual frames of the video will be passed into the model for sports ball detection, and and a bounding box will be drawn around the resulting detection that exhibits the highest confidence. To be able to demonstrate reasoning over time, the videos will exhibit occlusion of the balls. Two scenarios will be tested here: occulusion of the ball by a different object (piece of paper), and occusion by the same object (multiple balls crossing paths). Measurements of the centroids of successful detections will be extracted from the object detector. This will be useful in the second part of the assignment: object tracking. For this task, we will be running Kalman Filters to keep track of the state of each instance of sports ball in the video. These Kalman Filters will be updated at each frame depending on if there is a recorded measurement from the object detector. When the objects are occluded, the Kalman Filters can use the stored state to make predictions. These predictions at each frame will be recorded and superimposed onto the video, demonstrating how Kalman Filters can be used to track objects over time.  


## Task 2: Object Detector (40 points)

In this task you will use a CNN-based object detector to bound box all `ball` instances in each frame. Because the educational value is  not object detection, you are allowed to use an object detector of your choice trained to distinguish the `ball` class. You are free to use a pre-trained model (eg on MS COCO that contains the class `sports ball` or train a model yourself.  Ensure that you explain thoroughly the code. 

In [1]:
import cv2
import numpy as np



#load our list of classes
classes_lst = []
with open('yolov3.txt', 'r') as cf:
    #read line by line and add class name to our list
    lines = cf.readlines()
    for line in lines:
        classes_lst.append(line.strip())
        
#load the YOLOv3 deepnet
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')


#takes output from deepnet and puts boundary boxes on the frame passed in
def fetch_boxes_and_confidence(outputs, frame):
    
    #initialize list to store bounding boxes
    boxes = []
    
    #initialize list to store our confidences
    confidences = []
    
    #initialize list to store our class ids
    cIDs = []
    
    #initialize list to store the centroids of our detections
    center = []
    
    #fetch height and width of the frame for bounding box calculation
    height_px, width_px, _ = frame.shape
    
    #make bounding boxes for each detection in each output
    for output in outputs:
        for detection in output:
            
            #fetch scores from our detection
            scores = detection[5:]
            #find indice that has highest score 
            cID = np.argmax(scores)
            #index back into scores to fetch confidence
            confidence = scores[cID]
            #only put bounding boxes on detections with confidence higher than 0.5
            if confidence > 0.5:
                
                #fetch the center X and Y for each detection
                center_X = int(detection[0] * width_px)
                center_Y = int(detection[1] * height_px)
                #fetch the width and height for each detection
                width = int(detection[2] * width_px)
                height = int(detection[3] * height_px)
                
                #calculate the bottom left corner of the box
                x = int(center_X - width/2)
                y = int(center_Y - height/2)
                
                #add the box dimensional information to our list
                boxes.append([x,y,width,height])
                
                #store our confidence and class id
                confidences.append(confidence)
                cIDs.append(cID)
                
    #use non-max-suppresion on our boxes to limit detections
    chosen_boxes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    #put remaining boundings boxes on image 
    for idx in chosen_boxes:
        
        #get class label
        class_label = classes_lst[cIDs[idx]]
        
        #if not detecting sports ball, toss
        if class_label != "sports ball":
            continue
            
        #fetch box
        b = boxes[idx]
        #get coordinates and dimensional info for box
        x,y,w,h = b[0], b[1], b[2], b[3]
        
        #store the centroid 
        center.append([x + w/2, y + h/2])
        
        #put bounding box on image along with relevant information 
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame,class_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, color = (0,0,255))
    
    return frame, center
        
#use YOLOv3 to run detection on file   
def run_detection(filename):
    #create a videocapture object for the file
    capture = cv2.VideoCapture(filename)
    
    #store list of all centroids
    centroids = []
    
    #fetch frame width and height to get frame size
    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    frame_size = (frame_width,frame_height)
    #initialize our video
    vid = cv2.VideoWriter('%s_detection.avi' % filename[:-4], cv2.VideoWriter_fourcc(*'MJPG'),15, frame_size)
    
    #keep reading frames until no more frames can be read
    while True:
        
        #read a frame
        ret, frame = capture.read()
        
        #if we have anything to read
        if ret:
            
            #create blob object from the frame
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
            
            #initialze our input to the net with the blob
            net.setInput(blob)
            
            #get outputlayers
            layernames = net.getLayerNames()
            outputlayers = [layernames[i-1] for i in net.getUnconnectedOutLayers()]
            
            #net forward and get output (detections)
            output = net.forward(outputlayers)
            
            #put bounding boxes on our image based on the net output
            frame, center = fetch_boxes_and_confidence(output, frame)
            
            #store our centers from the output
            centroids.append(center)

            # Display the resulting frame
            cv2.imshow('frame', frame)
            
            #write the frame to a video
            vid.write(frame)

        else: 
            break
            
    # Release the capture
    capture.release()
    vid.release()
    cv2.destroyAllWindows()
    return centroids

#call our detection function on test videos
centroids_ball = run_detection("oneball.mp4")
centroids_multiball = run_detection("multiball.avi")

In [2]:
#Centroids for 1 ball
print(centroids_ball)

[[[905.5, 277.5]], [[907.0, 278.5]], [[905.5, 279.0]], [[905.5, 278.5]], [[906.5, 277.5]], [[905.5, 278.0]], [[906.5, 275.0]], [[906.5, 275.0]], [[906.5, 274.0]], [[892.5, 272.0]], [[861.5, 270.5]], [[830.5, 272.5]], [[808.0, 268.0]], [[782.5, 271.5]], [[758.5, 272.0]], [[740.0, 270.5]], [[712.5, 271.0]], [[689.5, 267.0]], [[666.5, 267.0]], [], [], [], [], [], [], [], [], [], [], [], [], [[358.0, 261.0]], [[342.0, 257.5]], [], [], [[301.5, 254.0]], [[281.5, 253.0]], [[263.5, 254.0]], [[242.5, 253.0]], [[224.0, 252.5]], [[203.0, 250.5]], [[185.5, 251.0]], [[163.5, 250.5]], [[145.0, 251.0]], [[129.5, 247.0]], [[113.5, 246.0]], [[96.0, 246.0]], [[82.0, 245.0]], [[67.0, 245.0]], [[53.5, 246.5]], [[45.5, 255.0]]]


In [3]:
#Centroids for 2 balls
print(centroids_multiball)

[[[621.5, 149.0]], [[621.5, 149.0]], [[621.5, 148.5]], [[622.0, 148.5]], [[619.5, 148.5]], [], [[603.0, 153.5]], [[585.5, 153.0]], [[564.0, 153.0]], [[539.5, 152.5]], [[518.0, 153.5], [26.0, 148.0]], [[494.5, 153.0], [46.0, 149.5]], [[472.5, 153.0], [71.0, 151.0]], [[451.0, 153.0], [104.5, 152.5]], [[430.5, 153.5], [136.0, 153.0]], [[409.0, 154.0], [163.5, 152.5]], [[389.0, 154.0], [193.0, 153.5]], [[368.0, 154.0], [219.5, 154.0]], [[349.0, 155.0], [250.5, 153.0]], [], [[309.5, 154.5]], [], [[364.5, 154.0], [269.5, 155.5]], [[391.5, 157.0], [253.5, 156.0]], [[421.0, 155.5], [234.5, 156.0]], [[449.0, 155.0], [218.0, 154.5]], [[475.5, 154.5], [198.0, 154.5]], [[503.5, 155.0], [182.0, 154.5]], [[531.0, 155.5], [163.5, 154.0]], [[556.0, 155.0], [145.5, 154.5]], [[128.5, 154.5], [580.5, 154.0]], [[108.5, 153.5], [596.0, 152.5]], [[92.5, 155.0]], [[74.5, 153.5], [619.5, 154.5]], [[56.5, 153.5]], [[44.0, 154.0]], [[34.5, 153.5]], [[26.5, 154.5]], [[19.5, 154.5]], [], []]


## Task 3: Tracker (40 points)

The detector outputs can be used to obtain the centroid(s) of the `ball` instances across time. You can assign a suitable starting state in the 1st frame of the video and obtain the predicted trajectory of the object during both visible and occluded frames. You need to superpose your predicted position of the object in each frame and the raw frame and store a sequence of all frames (generate a video).  Ensure that you explain thoroughly the code. 

```{note}
You can use OpenCV (`import cv2`) for only the satellite parts of this assignment - Use numpy, or better, jax to code the Kalman filter. You need to submit the assignment either as a notebook URL or a Github URL. 
```

In [4]:
#create our Kalman Filter Class
class KalmanFilter():
    
    def __init__(self, X, dt):
        
        self.X = X
        #initialize state transition matrix
        self.F = np.array([[1,0,dt,0],
                           [0,1,0,dt],
                           [0,0,1,0],
                           [0,0,0,1]])
        #observation model
        self.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
        #covariance of observation noise
        self.R = np.identity(2)
        #covariance of process noise
        self.Q = np.identity(4)
        #posteriori estimate covariance matrix
        self.P = np.identity(4)
        #control input model & control vector, none for this case
        self.B = 0
        self.U = 0
        
    def predict(self):
        #predict state estimate
        self.X = np.dot(self.F,self.X) + np.dot(self.B,self.U)
        #predict estimate covariance
        self.P = np.dot(np.dot(self.F, self.P),self.F.T) + self.Q

    def update_KF(self, z):
        #innovation covariance
        S = np.dot(np.dot(self.H,self.P), self.H.T) + self.R
        #calculate kalman gain
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        #update estimate covariance matrix
        self.P = np.dot(np.identity(len(K)) - np.dot(K,self.H), self.P)
        #update state estimate
        self.X = self.X + np.dot(K, z - np.dot(self.H,self.X))

        
#update our Kalman Filter parameters
def update(kf1,curr_center, frame, n_balls, kf2 = None):
    #if we only have 1 ball in image
    if n_balls == 1: 
        
        #fetch actual center of ball
        curr_center = np.array(curr_center).T
        
        #initial predict 
        kf1.predict()  
        
        #if we have a centroid from detection to use then update kalman filter
        if len(curr_center) != 0:
            kf1.update_KF(curr_center)
        
        #fetch prediction after updating
        pred_x, pred_y = kf1.X[0][0], kf1.X[1][0]
        
        #create coordinates based on prediction and draw circle on frame
        coords = (int(pred_x), int(pred_y))
        frame = cv2.circle(frame, coords, radius = 5, color = (0, 0, 255), thickness = 2)
        
        return kf1, None, frame
    else: # if we have 2 balls to track
        
        #predict on both filters
        kf1.predict()
        kf2.predict()
        
        #if we have centroids to sue
        if len(curr_center) != 0:
            #for each (max 2) centroid update the correct kalman filter
            for point in curr_center:
                
                #get one centroid
                p = np.array([point[0], point[1]])
                
                #get predictions from kalman filters
                center_1 = kf1.X[0][0], kf1.X[1][0]
                center_2 = kf2.X[0][0], kf2.X[1][0]
                
                #calculate euclidean distance of current centroid to each prediction
                euclid1 =  np.linalg.norm(p - center_1)
                euclid2 = np.linalg.norm(p - center_2)
                
                c = np.array([point]).T
                
                #update the kalman filter that is closer to the centroid
                if euclid1 < euclid2:
                    kf1.update_KF(c)
                else:
                    kf2.update_KF(c)
        
        #fetch predictions from both filters and create coordinates
        pred_x1, pred_y1 = kf1.X[0][0], kf1.X[1][0]
        coords1 = (int(pred_x1), int(pred_y1))
        pred_x2, pred_y2 = kf2.X[0][0], kf2.X[1][0]
        coords2 = (int(pred_x2), int(pred_y2))
        
        #update our frame with circle drawn on each coordinate
        frame = cv2.circle(frame, coords1, radius = 5, color = (0, 0, 255), thickness = 2)
        frame = cv2.circle(frame, coords2, radius = 5, color = (0, 255, 0), thickness = 2)
        
        return kf1, kf2, frame   
        
#perform tracking on video for 1 or 2 using kalman filters 
def tracking(filename, centroids, n_balls):
    
    #create videocapture object from file
    capture = cv2.VideoCapture(filename)
    #fetch frame width and height to create frame size
    frame_width = int(capture.get(3))
    frame_height = int(capture.get(4))
    frame_size = (frame_width,frame_height)
    #create video object
    vid = cv2.VideoWriter('%s_tracking.avi' % filename[:-4], cv2.VideoWriter_fourcc(*'MJPG'),15, frame_size)
    
    #if we only have 1 ball to track, initialize our kalman filter with the starting center position
    if n_balls == 1:
        #np array of position X, Y, velocity of X, and velocity of Y
        X = np.array([[centroids[0][0][0]], [centroids[0][0][1]], [1], [1]])
        kf1 = KalmanFilter(X, 1)
        kf2 = None
    else: #if we have two balls to track, initialize two kalman filters with initial center positions for first frame
        X1 = np.array([[621.5], [149.0], [1], [1]])
        kf1 = KalmanFilter(X1, 1)
        X2 = np.array([[5.0], [148.0], [1], [1]])
        kf2 = KalmanFilter(X2, 1)
    
    #counter for fetching centroids
    i = 0
    while True:
        #read a frame from the video
        ret, frame = capture.read()
        #if we have a frame to read
        if ret:
            
            #fetch center(s)
            curr_center = centroids[i]  
            
            #update our kalman filters and get updated frame with tracker
            kf1, kf2, frame = update(kf1, curr_center, frame,n_balls, kf2)
            
            #write frame into video
            vid.write(frame)
            i+=1
                
        else: 
            break
    #Release the capture
    capture.release()
    vid.release()
    cv2.destroyAllWindows()
    
#call our tracking function on test videos
tracking("oneball.mp4", centroids_ball, 1)
tracking("multiball.avi", centroids_multiball, 2)